In [ ]:
# !git clone https://github.com/serengil/deepface.git

In [ ]:
# pip install deepface
# pip install tf-keras


import os
import cv2 as cv
import random
import shutil
import math
import numpy as np
import matplotlib.pyplot as plt
from deepface import DeepFace

In [ ]:
import os

os.environ["USERPROFILE"] = "G:/grey_sciml/deepface"  # This works better on Windows
os.environ["HOME"] = "G:/grey_sciml/deepface"
print("DeepFace directory:", os.path.expanduser("~/.deepface"))

In [ ]:
img = cv.imread(r'G:\grey_sciml\frame_faces\dist\26_Dheeraj_37.jpg')
# img = cv.resize(img,(200,250))
plt.imshow(img)
plt.show()

In [ ]:
# Angular distance matrix for image registration
def angular_distance(source_representation, test_representation):

    # Normalize the vectors
    source_normalized = source_representation / np.linalg.norm(source_representation)
    test_normalized = test_representation / np.linalg.norm(test_representation)
    
    # Calculate cosine similarity
    cosine_similarity = np.dot(source_normalized, test_normalized)
    
    # Ensure cosine similarity is within valid range [-1, 1]
    cosine_similarity = np.clip(cosine_similarity, -1.0, 1.0)
    
    # Calculate angular distance
    angular_dist = np.arccos(cosine_similarity) / np.pi
    
    return angular_dist

## Face recogination in Images

In [ ]:
def face_recogination(path):
    
    fig,ax = plt.subplots(nrows=3,figsize=(6,12))
    print("path",path)
    name = os.path.splitext(os.path.basename(path))[0]
    img = cv.imread(path)
    img_rgb = cv.cvtColor(img , cv.COLOR_BGR2RGB)
    ax[0].imshow(img_rgb)
    ax[0].set_title(f"original Input Image-{name}")

    detected_face = DeepFace.extract_faces(path, detector_backend='ssd')
    x,y,w,h,_,_ = detected_face[0]['facial_area'].values()
    x1,y1 = x,y
    x2,y2 = x+w,y+h
    cv.rectangle(img_rgb,(x1,y1),(x2,y2),(0,0,0),2)
    ax[1].imshow(img_rgb)
    cropped_img = img[y1:y2, x1:x2]

    cv.imwrite("Cropped.jpg",cropped_img )
    dfs = DeepFace.find(img_path = r"G:\grey_sciml\Cropped.jpg",db_path = r"G:\grey_sciml\db",model_name = "ArcFace")
    det_path = dfs[0]['identity'][0]
    name = os.path.splitext(os.path.basename(det_path))[0]
    det_img = cv.imread(det_path)
    det_img_rgb = cv.cvtColor(det_img , cv.COLOR_BGR2RGB)
    ax[2].imshow(det_img_rgb)
    ax[2].set_title(f"Selected_img-{name}")


face_recogination(r"db/Alvaro Morte244_235.jpg")

## Face recogination in in video

In [ ]:
def face_reco_video(video_path, output_path, db_path):
    video = cv.VideoCapture(video_path)       # video_path : path to the video file or 0 for the default camera

    fwidth, fheight = int(video.get(cv.CAP_PROP_FRAME_WIDTH)), int(video.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv.CAP_PROP_FPS))    # get the FPS of the video
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter(output_path, fourcc, fps, (fwidth, fheight))
    frame_count = 0
    # print(fps)
    while True:
        ret, frame = video.read()
        if not ret:
            break

        img_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        # cv.imshow("Frame",img_rgb)
        try:
            # face detection
            detected_faces = DeepFace.extract_faces(frame, 
                                                  detector_backend='yolov8',
                                                #   enforce_detection=False
                                                   )                         

            for face in detected_faces:
                if face['confidence']>0.7:
                    x, y, w, h, _, _ = face['facial_area'].values()
                    cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    cropped_img = frame[y:y+h, x:x+w]
                    # temp_path =q "temp_face.jpg"
                    # cv.imwrite(temp_path, cropped_img)
                    amount=1.5
                    blurred = cv.GaussianBlur(cropped_img,(5,5),5)
                    gray = cv.cvtColor(blurred, cv.COLOR_BGR2GRAY)
                    laplacian = cv.Laplacian(gray, cv.CV_64F)
                    variance = laplacian.var()
                    print(variance)
                    if variance>10:
                        
                        # sharpened = float(amount + 1) * cropped_img - float(amount) * blurred
                        # sharpened = np.clip(sharpened, 0, 255).astype(np.uint8)

                        # # fig, ax = plt.subplots(nrows =1,ncols=2)
                        # sharpened = cv.cvtColor(sharpened,cv.COLOR_BGR2RGB)
                        # # sharpened = cv.cvtColor(sharpened,cv.COLOR_BGR2RGB)
                        # # out_img = deblur(cropped_img)
                        # out_img = cv.cvtColor(sharpened,cv.COLOR_BGR2RGB)
                        try:
                            # Face recognition
                            dfs = DeepFace.find(img_path=cropped_img, 
                                                db_path=db_path, 
                                                model_name="ArcFace",
                                                #   enforce_detection=False, 
                                                distance_metric= 'cosine',
                                                silent=True
                                                )
                            # print(dfs)
                            # text putting in the frame and showing the result
                            if len(dfs) > 0 and len(dfs[0]) > 0:
                                det_path = dfs[0]['identity'][0]
                                name = det_path.split("\\")[3]
                                cv.imwrite(f"G:/grey_sciml/frame_faces/dist/{frame_count}_{name}_{int(variance)}.jpg",cropped_img)
                                cv.putText(frame, f"frame-{frame_count}", (10,20 ), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                                conf = round(1 - dfs[0]["distance"][0], 3) 
                                cv.putText(frame, f"{name},conf-{conf}", (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                        except:
                            print(f"Not detected_{frame_count}")
                            pass

        except:
            print(f"Not recoginised_{frame_count}")
            pass

        out.write(frame)

        cv.imshow("Face Recognition", frame)  # Show live result
        frame_count += 1
        if cv.waitKey(60) & 0xFF == ord('q'):  # Press 'q' to exit
            break

        # Release resources
    video.release()
    out.release()
    cv.destroyAllWindows()
    print("completed")

In [ ]:
face_reco_video(
    video_path="test_dist.mp4",
    output_path=r"G:\grey_sciml\out_dist.mp4",
    db_path=r"G:\grey_sciml\vedio_db"
)